In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Melanoma"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Melanoma/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Melanoma/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Melanoma/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Melanoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
import os
import pandas as pd

# Review subdirectories to find the most relevant match for Melanoma
all_dirs = os.listdir(tcga_root_dir)

# Print all available directories for debugging
print("Available TCGA directories:")
print(all_dirs)

# Looking for directories related to Melanoma
melanoma_related_dirs = [d for d in all_dirs if "melanoma" in d.lower()]

if len(melanoma_related_dirs) > 0:
    # If we found related directories, choose the most specific one
    # Between 'TCGA_Melanoma_(SKCM)' and 'TCGA_Ocular_melanomas_(UVM)', SKCM is more representative of melanoma in general
    if "TCGA_Melanoma_(SKCM)" in melanoma_related_dirs:
        selected_dir = "TCGA_Melanoma_(SKCM)"
    else:
        selected_dir = melanoma_related_dirs[0]
    
    selected_path = os.path.join(tcga_root_dir, selected_dir)
    
    # Get paths to the clinical and genetic data files
    clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(selected_path)
    
    # Load the data files
    clinical_data = pd.read_csv(clinical_file_path, index_col=0, sep='\t')
    genetic_data = pd.read_csv(genetic_file_path, index_col=0, sep='\t')
    
    # Print the column names of the clinical data
    print("Clinical data columns:")
    print(clinical_data.columns.tolist())
    
    # Also print basic information about both datasets
    print("\nClinical data shape:", clinical_data.shape)
    print("Genetic data shape:", genetic_data.shape)
    
    # Set flags for validation
    is_gene_available = genetic_data.shape[0] > 0
    is_trait_available = clinical_data.shape[0] > 0
else:
    print("No directories found related to Melanoma in the TCGA dataset.")
    
    # Mark this task as completed with no suitable directory found
    is_gene_available = False
    is_trait_available = False
    validate_and_save_cohort_info(
        is_final=False, 
        cohort="TCGA", 
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available
    )


Available TCGA directories:
['TCGA_Liver_Cancer_(LIHC)', 'TCGA_Lower_Grade_Glioma_(LGG)', 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)', 'TCGA_Lung_Adenocarcinoma_(LUAD)', 'TCGA_Lung_Cancer_(LUNG)', 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)', 'TCGA_Melanoma_(SKCM)', 'TCGA_Mesothelioma_(MESO)', 'TCGA_Ocular_melanomas_(UVM)', 'TCGA_Ovarian_Cancer_(OV)', 'TCGA_Pancreatic_Cancer_(PAAD)', 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)', 'TCGA_Prostate_Cancer_(PRAD)', 'TCGA_Rectal_Cancer_(READ)', 'TCGA_Sarcoma_(SARC)', 'TCGA_Stomach_Cancer_(STAD)', 'TCGA_Testicular_Cancer_(TGCT)', 'TCGA_Thymoma_(THYM)', 'TCGA_Thyroid_Cancer_(THCA)', 'TCGA_Uterine_Carcinosarcoma_(UCS)', '.DS_Store', 'CrawlData.ipynb', 'TCGA_Acute_Myeloid_Leukemia_(LAML)', 'TCGA_Adrenocortical_Cancer_(ACC)', 'TCGA_Bile_Duct_Cancer_(CHOL)', 'TCGA_Bladder_Cancer_(BLCA)', 'TCGA_Breast_Cancer_(BRCA)', 'TCGA_Cervical_Cancer_(CESC)', 'TCGA_Colon_and_Rectal_Cancer_(COADREAD)', 'TCGA_Colon_Cancer_(COAD)', 'TCGA_Endometrioid_Cancer_

Clinical data columns:
['_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'age_at_initial_pathologic_diagnosis', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'breslow_depth_value', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_additional_surgery_procedure', 'days_to_new_tumor_event_after_initial_treatment', 'days_to_submitted_specimen_dx', 'distant_metastasis_anatomic_site', 'followup_case_report_form_submission_reason', 'form_completion_date', 'gender', 'height', 'history_of_neoadjuvant_treatment', 'icd_10', 'icd_o_3_histology', 'icd_o_3_site', 'informed_consent_verified', 'initial_weight', 'interferon_90_day_prior_excision_admin_indicator', 'is_ffpe', 'lactate_dehydrogenase_result', 'lost_follow_up', 'malignant_neoplasm_mitotic_count_rate', 'melanoma_clark_level_value'

### Step 2: Find Candidate Demographic Features

In [3]:
# Identify possible age and gender columns from clinical data
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth']
candidate_gender_cols = ['gender']

# Prepare to display the data
import pandas as pd
import os

cohort_dir = os.path.join(tcga_root_dir, "TCGA_Melanoma_(SKCM)")
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
clinical_df = pd.read_csv(clinical_file_path, index_col=0, sep='\t')

# Preview age columns
if candidate_age_cols:
    age_preview = {}
    for col in candidate_age_cols:
        if col in clinical_df.columns:
            age_preview[col] = clinical_df[col].head(5).tolist()
    print("Age columns preview:")
    print(age_preview)

# Preview gender columns
if candidate_gender_cols:
    gender_preview = {}
    for col in candidate_gender_cols:
        if col in clinical_df.columns:
            gender_preview[col] = clinical_df[col].head(5).tolist()
    print("\nGender columns preview:")
    print(gender_preview)


Age columns preview:
{'age_at_initial_pathologic_diagnosis': [71.0, 82.0, 82.0, 46.0, 74.0], 'days_to_birth': [-26176.0, -30286.0, -30163.0, -17025.0, -27124.0]}

Gender columns preview:
{'gender': ['MALE', 'MALE', 'MALE', 'FEMALE', 'FEMALE']}


### Step 3: Select Demographic Features

In [4]:
# Review age column candidates
age_candidates = {
    'age_at_initial_pathologic_diagnosis': [71.0, 82.0, 82.0, 46.0, 74.0], 
    'days_to_birth': [-26176.0, -30286.0, -30163.0, -17025.0, -27124.0]
}

# Review gender column candidates
gender_candidates = {
    'gender': ['MALE', 'MALE', 'MALE', 'FEMALE', 'FEMALE']
}

# Select the appropriate columns
# For age, 'age_at_initial_pathologic_diagnosis' is directly in years and easier to interpret
age_col = 'age_at_initial_pathologic_diagnosis' if 'age_at_initial_pathologic_diagnosis' in age_candidates else None

# For gender, there's only one candidate column and it contains valid values
gender_col = 'gender' if 'gender' in gender_candidates else None

# Print the selected columns
print(f"Selected age column: {age_col}")
print(f"Selected gender column: {gender_col}")


Selected age column: age_at_initial_pathologic_diagnosis
Selected gender column: gender


### Step 4: Feature Engineering and Validation

In [5]:
# Let's use the correct folder for Melanoma data
selected_dir = "TCGA_Melanoma_(SKCM)"
selected_path = os.path.join(tcga_root_dir, selected_dir)

# Reload the clinical and genetic data files to ensure we're using the correct dataset
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(selected_path)
clinical_df = pd.read_csv(clinical_file_path, index_col=0, sep='\t')
genetic_df = pd.read_csv(genetic_file_path, index_col=0, sep='\t')

# 1. Extract and standardize clinical features (trait, age, gender)
selected_clinical_df = tcga_select_clinical_features(
    clinical_df, 
    trait=trait, 
    age_col=age_col, 
    gender_col=gender_col
)

# 2. Normalize gene symbols in gene expression data
normalized_gene_df = normalize_gene_symbols_in_index(genetic_df)

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_df.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")
print(f"Gene data shape after normalization: {normalized_gene_df.shape}")

# 3. Link clinical and genetic data
# Transpose the genetic data to have samples as rows
genetic_df_t = normalized_gene_df.T
# Ensure the indices match between datasets
common_samples = list(set(genetic_df_t.index) & set(selected_clinical_df.index))
genetic_df_filtered = genetic_df_t.loc[common_samples]
clinical_df_filtered = selected_clinical_df.loc[common_samples]

# Combine the datasets
linked_data = pd.concat([clinical_df_filtered, genetic_df_filtered], axis=1)
print(f"Linked data shape: {linked_data.shape}")

# 4. Handle missing values in the linked data
linked_data_cleaned = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data_cleaned.shape}")

# 5. Determine if trait and demographic features are biased
is_biased, linked_data_filtered = judge_and_remove_biased_features(linked_data_cleaned, trait)

# 6. Validate data quality and save cohort information
# First check if gene and trait data are available
is_gene_available = linked_data_filtered.shape[1] > 3  # More columns than just trait, age, gender
is_trait_available = trait in linked_data_filtered.columns

# Second validation for saving metadata
note = f"Dataset contains {linked_data_filtered.shape[0]} samples and {linked_data_filtered.shape[1] - 3} genes after preprocessing."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data_filtered,
    note=note
)

# 7. Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_filtered.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
    
    # Also save the clinical data separately
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_cols = [col for col in linked_data_filtered.columns if col in [trait, 'Age', 'Gender']]
    linked_data_filtered[clinical_cols].to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
else:
    print("Dataset not usable for analysis. Data not saved.")

Normalized gene data saved to ../../output/preprocess/Melanoma/gene_data/TCGA.csv
Gene data shape after normalization: (19848, 474)
Linked data shape: (474, 19851)


Data shape after handling missing values: (474, 19851)
For the feature 'Melanoma', the least common label is '0' with 1 occurrences. This represents 0.21% of the dataset.
The distribution of the feature 'Melanoma' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 48.0
  50% (Median): 58.0
  75%: 70.75
Min: 15.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 180 occurrences. This represents 37.97% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Dataset not usable for analysis. Data not saved.
